In [1]:
import pandas as pd
import pandas_ta as ta
import config
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.orders as orders

from datetime import datetime, timezone
import time

In [2]:
# Setup your OANDA connection
client = API(access_token=config.OANDA_API_KEY)

In [3]:
# define variables
timeframe = "M5"
instrument = "GBP_JPY"

In [4]:
def get_candles(tf):
    params = {
        "granularity": tf,
        "price": "A"  # Ask prices
    }

    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    candles = client.request(r)['candles']

    # Convert to pandas DataFrame
    data = []

    for c in candles:
        if c["complete"]:
            data.append({
                "time": c["time"],
                "open": float(c["ask"]["o"]),
                "high": float(c["ask"]["h"]),
                "low": float(c["ask"]["l"]),
                "close": float(c["ask"]["c"])
            })

    df = pd.DataFrame(data)
    df["time"] = pd.to_datetime(df["time"])
    
    return df

In [5]:

# --- Breakout indicators (Donchian + ATR) ---
def calculate_indicators(df, donchian_len=20, atr_len=14):
    """
    Extends the existing indicators with Donchian Channel levels for breakout entries.
    Expects columns: ['open','high','low','close'] (and optionally 'volume').
    """
    # Keep existing EMAs if ta is available (backward compatible with the EMA strategy)
    try:
        df["EMA_5"] = ta.ema(df["close"], length=5)
        df["EMA_8"] = ta.ema(df["close"], length=8)
    except Exception:
        pass

    # ATR for volatility-based stops
    try:
        df["ATR_14"] = ta.atr(df["high"], df["low"], df["close"], length=atr_len)
    except Exception:
        # Fallback ATR if ta isn't available (True Range + SMA)
        tr1 = (df["high"] - df["low"]).abs()
        tr2 = (df["high"] - df["close"].shift(1)).abs()
        tr3 = (df["low"] - df["close"].shift(1)).abs()
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        df["ATR_14"] = tr.rolling(atr_len, min_periods=atr_len).mean()

    # Donchian Channel (exclude the current bar for clean breakouts)
    dc_h = df["high"].rolling(donchian_len, min_periods=donchian_len).max().shift(1)
    dc_l = df["low"].rolling(donchian_len, min_periods=donchian_len).min().shift(1)
    df["DC_HIGH"] = dc_h
    df["DC_LOW"]  = dc_l

    # Optional volume filter support (SMA of volume)
    if "volume" in df.columns:
        df["VOL_SMA_20"] = df["volume"].rolling(20, min_periods=20).mean()

    return df


In [6]:
# Function to place an order
def place_order(stop_loss, take_profit):
    data = {
        "order": {
            "instrument": instrument,
            "units": 1,
            "type": "MARKET",
            "stopLossOnFill": {"price": f"{stop_loss:.3f}"},
            "takeProfitOnFill": {"price": f"{take_profit:.3f}"},
        }
    }

    r = orders.OrderCreate(config.OANDA_ACCOUNT_ID, data=data)
    client.request(r)
    print(f"Placed order for {instrument} with stop loss at {round(stop_loss, 3)} and take profit at {round(take_profit, 3)}.")

In [7]:

# --- Donchian/ATR Breakout Strategy ---
def breakout_strategy(df, donchian_len=20, atr_mult=1.0, rr=2.0, use_volume_filter=False):
    """
    Long-only breakout:
      Entry when close crosses above prior N-bar highest high (Donchian upper band).
      Stop = entry - atr_mult * ATR_14
      Take-profit = entry + rr * (entry - stop)
    """
    if len(df) < max(donchian_len + 2, 30):
        print("Not enough data for breakout strategy")
        return

    last = df.iloc[-1]
    prev = df.iloc[-2]

    # Basic breakout condition
    breakout_up = (prev["close"] <= prev["DC_HIGH"]) and (last["close"] > last["DC_HIGH"])

    # Optional volume filter: current volume above its 20-SMA
    vol_ok = True
    if use_volume_filter and "VOL_SMA_20" in df.columns and not pd.isna(last.get("VOL_SMA_20", None)):
        vol_ok = last.get("volume", 0) > last["VOL_SMA_20"]

    if breakout_up and vol_ok:
        entry = float(last["close"])
        atr = float(last["ATR_14"]) if not pd.isna(last["ATR_14"]) else float(df["ATR_14"].dropna().iloc[-1])
        stop_loss = entry - atr_mult * atr
        take_profit = entry + rr * (entry - stop_loss)
        print(f"Buy Signal: Breakout above {round(last['DC_HIGH'], 5)} | Entry={entry:.5f} SL={stop_loss:.5f} TP={take_profit:.5f}")
        place_order(stop_loss, take_profit)
    else:
        print("Strategy conditions not met")


In [8]:
def run_bot():
    print("Starting trading bot")
    last_checked = None

    while True:
        current_time = datetime.now(timezone.utc)
        
        # Check for a new 15 minute candle
        #if current_time.minute % 15 == 0 and current_time.second < 10:
        if current_time.minute % 5 == 0 and current_time.second < 10:
            # Check if the 15-minute interval has changed since the last check
            if last_checked != current_time.minute:
                print(f"Current time: {current_time}")
                print("Checking for trade signals")
                price = get_candles(timeframe)
                price = calculate_indicators(price)
                breakout_strategy(price)
                last_checked = current_time.minute  # Update to prevent re-triggering

        time.sleep(1)

In [9]:
run_bot()

Starting trading bot
Current time: 2025-11-05 09:40:00.059242+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:45:00.492934+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:50:00.049703+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:55:00.978601+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 10:00:00.489753+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 10:10:00.780969+00:00
Checking for trade signals
Strategy conditions not met


KeyboardInterrupt: 